1. Obtener lista de antivirales con scrapeo de BD
2. Crear una columna de SMILES de esos antivirales
3. Pasar la columna de SMILES a la función de maic con selenium para obtener la afinidad
4. Crear una columna con esos valores

In [1]:
import pubchempy as pcp
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup

In [2]:
def getSmilesByCID(cid):
    c = pcp.Compound.from_cid(cid)
    return c.canonical_smiles

def getSmilesByName(name):
    for compound in pcp.get_compounds(name, 'name'):
        return compound.canonical_smiles

In [4]:
page_number = 1

nombres = []
codigos = []

while page_number < 25:
    
    page_number += 1
    
    url = "https://www.drugbank.ca/unearth/q?approved=1&c=_score&ca=0&d=down&eu=0&page={}&query=antiviral&searcher=drugs&us=0".format(page_number)
    data = requests.get(url).text
    soup = BeautifulSoup(data, 'html.parser')
    hits = soup.find_all("h2",{"class":"hit-link"})
    
    for i in hits:
        #Crear la lista con sólo los nombres
        match = re.findall(r'DB\d{5}..(.*)</a></h2>', str(i))
        nombres.append(match)
        #Crear la lista con sólo los códigos
        db_code = re.findall(r'DB\d{5}', str(i))
        codigos.append(db_code)


In [5]:
print(len(nombres))
print(len(codigos))

81
81


In [6]:
df = pd.DataFrame(list(zip(nombres, codigos)))
df.head()

,0,1
0,[Emtricitabine],[DB00879]
1,[Zidovudine],[DB00495]
2,[Darunavir],[DB01264]
3,[Nevirapine],[DB00238]
4,[Valganciclovir],[DB01610]


In [7]:
#df.to_csv("output/antivirales.csv")

In [8]:
df.columns = ["nombre", "código"]

In [9]:
df['SMILES']=df.apply(lambda x: getSmilesByName(x.nombre), axis=1)

In [10]:
df.head()

,nombre,código,SMILES
0,[Emtricitabine],[DB00879],C1C(OC(S1)CO)N2C=C(C(=NC2=O)N)F
1,[Zidovudine],[DB00495],CC1=CN(C(=O)NC1=O)C2CC(C(O2)CO)N=[N+]=[N-]
2,[Darunavir],[DB01264],CC(C)CN(CC(C(CC1=CC=CC=C1)NC(=O)OC2COC3C2CCO3)...
3,[Nevirapine],[DB00238],CC1=C2C(=NC=C1)N(C3=C(C=CC=N3)C(=O)N2)C4CC4
4,[Valganciclovir],[DB01610],CC(C)C(C(=O)OCC(CO)OCN1C=NC2=C1N=C(NC2=O)N)N


In [3]:
#df.to_csv("output/antiviralesconsmiles.csv")

In [40]:
df = pd.read_csv("output/antiviralesconsmiles.csv")

In [46]:
df.dropna(inplace=True)
df.columns
#df.drop("Unnamed: 0", axis=1, inplace=True)

df

,nombre,código,SMILES
0,['Emtricitabine'],['DB00879'],C1C(OC(S1)CO)N2C=C(C(=NC2=O)N)F
1,['Zidovudine'],['DB00495'],CC1=CN(C(=O)NC1=O)C2CC(C(O2)CO)N=[N+]=[N-]
2,['Darunavir'],['DB01264'],CC(C)CN(CC(C(CC1=CC=CC=C1)NC(=O)OC2COC3C2CCO3)...
3,['Nevirapine'],['DB00238'],CC1=C2C(=NC=C1)N(C3=C(C=CC=N3)C(=O)N2)C4CC4
4,['Valganciclovir'],['DB01610'],CC(C)C(C(=O)OCC(CO)OCN1C=NC2=C1N=C(NC2=O)N)N
...,...,...,...
76,['Lamivudine'],['DB00709'],C1C(OC(S1)CO)N2C=CC(=NC2=O)N
77,['Atazanavir'],['DB01072'],CC(C)(C)C(C(=O)NC(CC1=CC=CC=C1)C(CN(CC2=CC=C(C...
78,['Rilpivirine'],['DB08864'],CC1=CC(=CC(=C1NC2=NC(=NC=C2)NC3=CC=C(C=C3)C#N)...
79,['Delavirdine'],['DB00705'],CC(C)NC1=C(N=CC=C1)N2CCN(CC2)C(=O)C3=CC4=C(N3)...


In [42]:
## FASTAS

proteinas = pd.read_csv("output/55_FASTA.csv")
proteinas.head()

,id_cadena,FASTA
0,6YB7_A,SGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTS...
1,6W75_D,SNAAGNATEVPANSTVLSFCAFAVDAAKAYKDYLASGGQPITNCVK...
2,6W75_C,SNASSQAWQPGVAMPNLYKMQRMLLEKCDLQNYGDSATLPKGIMMN...
3,6W75_B,SNAAGNATEVPANSTVLSFCAFAVDAAKAYKDYLASGGQPITNCVK...
4,6W75_A,SNASSQAWQPGVAMPNLYKMQRMLLEKCDLQNYGDSATLPKGIMMN...


In [49]:
data = pd.DataFrame()

for drug_index in df.transpose():
    for protein_index in proteinas.transpose():
        data = data.append(pd.DataFrame({'Nombre':df['nombre'][drug_index],
                                          'SMILES':df['SMILES'][drug_index],
                                          'id_cadena':proteinas['id_cadena'][protein_index],
                                          'FASTA':proteinas['FASTA'][protein_index]},index=[0]),ignore_index=True)
        
    
        

In [50]:
data.to_csv("output/data.csv")
data.head()

,Nombre,SMILES,id_cadena,FASTA
0,['Emtricitabine'],C1C(OC(S1)CO)N2C=C(C(=NC2=O)N)F,6YB7_A,SGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTS...
1,['Emtricitabine'],C1C(OC(S1)CO)N2C=C(C(=NC2=O)N)F,6W75_D,SNAAGNATEVPANSTVLSFCAFAVDAAKAYKDYLASGGQPITNCVK...
2,['Emtricitabine'],C1C(OC(S1)CO)N2C=C(C(=NC2=O)N)F,6W75_C,SNASSQAWQPGVAMPNLYKMQRMLLEKCDLQNYGDSATLPKGIMMN...
3,['Emtricitabine'],C1C(OC(S1)CO)N2C=C(C(=NC2=O)N)F,6W75_B,SNAAGNATEVPANSTVLSFCAFAVDAAKAYKDYLASGGQPITNCVK...
4,['Emtricitabine'],C1C(OC(S1)CO)N2C=C(C(=NC2=O)N)F,6W75_A,SNASSQAWQPGVAMPNLYKMQRMLLEKCDLQNYGDSATLPKGIMMN...


In [57]:
lst = []

for i in range(len(data)):
    dct = {
        "protein_ID": data.iloc[i]["id_cadena"],
        "FASTA": data.iloc[i]["FASTA"],
        "small_mol_name": data.iloc[i]["Nombre"][0],
        "SMILES": data.iloc[i]["SMILES"]}
    lst.append(dct)


In [59]:
'''
with open('output/data.txt', 'w') as file:
    for listitem in lst:
        file.write('%s\n' % listitem)
'''

"\nwith open('output/data.txt', 'w') as file:\n    for listitem in lst:\n        file.write('%s\n' % listitem)\n"

In [1]:
#print(lst)

## 2 protein Sample: 

6W41-C: Chain C, Spike glycoprotein receptor binding domain

6YB7-A: Chain A, SARS-CoV-2 main protease


In [60]:
sample = data[(data["id_cadena"] == "6W41_C") | (data["id_cadena"] == "6YB7_A")]

sample.shape

(138, 4)

In [73]:
lst = []

for i in range(len(sample)):
    dct = {
        "protein_ID": sample.iloc[i]["id_cadena"],
        "FASTA": sample.iloc[i]["FASTA"],
        "small_mol_name": sample.iloc[i]["Nombre"],
        "SMILES": sample.iloc[i]["SMILES"]}
    lst.append(dct)

In [74]:
print(lst)

[{'protein_ID': '6YB7_A', 'FASTA': 'SGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTSEDMLNPNYEDLLIRKSNHNFLVQAGLVQLRVIGHSMQNCVLKLKVDTANPKTPKYKFVRIQPGQTFSVLACYNGSPSGVYQCAMRPNFTIKGSFLNGSCGSVGFNIDYDCVSFCYMHHMELPTGVHAGTDLEGNFYGPFVDRQTAQAAGTDTTITVNVLAWLYAAVINGDRWFLNRFTTTLNDFNLVAMKYNYEPLTQDHVDILGPLSAQTGIAVLDMCASLKELLQNGMNGRTILGSALLEDEFTPFDVVRQCSGVTFQ', 'small_mol_name': "['Emtricitabine']", 'SMILES': 'C1C(OC(S1)CO)N2C=C(C(=NC2=O)N)F'}, {'protein_ID': '6W41_C', 'FASTA': 'RVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFSGHHHHHH', 'small_mol_name': "['Emtricitabine']", 'SMILES': 'C1C(OC(S1)CO)N2C=C(C(=NC2=O)N)F'}, {'protein_ID': '6YB7_A', 'FASTA': 'SGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTSEDMLNPNYEDLLIRKSNHNFLVQAGLVQLRVIGHSMQNCVLKLKVDTANPKTPKYKFVRIQPGQTFSVLACYNGSPSGVYQCAMRPNFTIKGSFLNGSCGSVGFNIDYDCVSFCYMHHMELPTGVHAGTDLEGNF

In [75]:
import random
subsample = []

for i in range(60):
    subsample.append(random.choice(lst))

In [79]:
print(subsample)

[{'protein_ID': '6W41_C', 'FASTA': 'RVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFSGHHHHHH', 'small_mol_name': "['Stavudine']", 'SMILES': 'CC1=CN(C(=O)NC1=O)C2C=CC(O2)CO'}, {'protein_ID': '6YB7_A', 'FASTA': 'SGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTSEDMLNPNYEDLLIRKSNHNFLVQAGLVQLRVIGHSMQNCVLKLKVDTANPKTPKYKFVRIQPGQTFSVLACYNGSPSGVYQCAMRPNFTIKGSFLNGSCGSVGFNIDYDCVSFCYMHHMELPTGVHAGTDLEGNFYGPFVDRQTAQAAGTDTTITVNVLAWLYAAVINGDRWFLNRFTTTLNDFNLVAMKYNYEPLTQDHVDILGPLSAQTGIAVLDMCASLKELLQNGMNGRTILGSALLEDEFTPFDVVRQCSGVTFQ', 'small_mol_name': "['Boceprevir']", 'SMILES': 'CC1(C2C1C(N(C2)C(=O)C(C(C)(C)C)NC(=O)NC(C)(C)C)C(=O)NC(CC3CCC3)C(=O)C(=O)N)C'}, {'protein_ID': '6YB7_A', 'FASTA': 'SGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTSEDMLNPNYEDLLIRKSNHNFLVQAGLVQLRVIGHSMQNCVLKLKVDTANPKTPKYKFVRIQPGQTFSVLACYNGSPSGVYQCAMRPNFTIKGSFLNGS

In [80]:
subsample[0]

{'protein_ID': '6W41_C',
 'FASTA': 'RVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVNFSGHHHHHH',
 'small_mol_name': "['Stavudine']",
 'SMILES': 'CC1=CN(C(=O)NC1=O)C2C=CC(O2)CO'}